In [ ]:
import sys
from mcap.reader import make_reader
from mcap_ros2.decoder import DecoderFactory, Decoder
from event_camera_py import Decoder as ECDecoder
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
from pathlib import Path
from event_camera_py import Decoder
from bag_reader_ros2 import BagReader
import copy
import yaml
import cv2

In [ ]:
BAG=Path("/data/ha_ec_data/2024.08.19.Pennov.Flight.and.Calib/flights/ha_ec_2024-08-19-12-11-57/ha_ec_2024-08-19-12-11-57_0.mcap")
VIDEO_PATH = BAG.parent/ "rgb_cam_video.avi"
RGB_IMAGES_PATH = BAG.parent/"rgb_images"
TRIGGER_FREQ = 50 # Hz

# FLIR-related constants
FLIR_TOPIC_RAW = "/cam_sync/cam0/image_raw"
FLIR_TOPIC_INFO =  "/cam_sync/cam0/camera_info"
FLIR_TOPIC_META = "/cam_sync/cam0/meta"

# Get statistics from the bag
with open(BAG, "rb") as f:
    reader = make_reader(f, decoder_factories=[DecoderFactory()])

    # Get a dictionnary of the channels
    ch = reader.get_summary().channels
    topics = {ch[idx].topic:idx for idx in ch}
    # pprint(topics)
    
    # Get channel for FLIR and number of messages
    stats = reader.get_summary().statistics

# Check if the video exists to avoid overwritting it
if VIDEO_PATH.exists():
    sys.exit(f"Video path {VIDEO_PATH} exists. Stopping execution")

# Create images folder if it does not exist
if not RGB_IMAGES_PATH.exists():
    RGB_IMAGES_PATH.mkdir(parents=True, exist_ok=True)
else:
    if RGB_IMAGES_PATH.is_dir() and any(RGB_IMAGES_PATH.iterdir()):
        sys.exit(f"RGB images path {RGB_IMAGES_PATH} is not empty. Stopping execution.")
    

In [ ]:
# Get number of channels for the FLIR camera
flir_channel_raw = topics[FLIR_TOPIC_RAW]
flir_channel_info = topics[FLIR_TOPIC_INFO]
flir_channel_meta = topics[FLIR_TOPIC_META]
flir_number_raw = stats.channel_message_counts[flir_channel_raw]
flir_number_info = stats.channel_message_counts[flir_channel_info]
flir_number_meta = stats.channel_message_counts[flir_channel_meta]


# Number of messages for both channels should be the same
try:
    assert (flir_number_raw == flir_number_info == flir_number_meta)
except AssertionError:
    print(f"The number of messages does not match:")
    print(f"flir_raw: {flir_number_raw}")
    print(f"flir_info: {flir_number_info}")
    print(f"flir_meta: {flir_number_meta}")

num_msgs = flir_number_raw
width = 2048
height = 1536

fourcc = cv2.VideoWriter_fourcc(*"MJPG")
video = cv2.VideoWriter(str(VIDEO_PATH), fourcc, 50.0, (width, height), True)

with open(BAG, "rb") as f:
    reader = make_reader(f, decoder_factories=[DecoderFactory()])

    factory = DecoderFactory()
    decoders = {}
    
    i = 0
    for schema, channel, encoded_msg in reader.iter_messages():
        if channel.id != flir_channel_raw:
            continue
        # Create decoder for the channel id if it does not exist
        if channel.id not in decoders:
            decoders[channel.id] = factory.decoder_for(channel.message_encoding, schema)
 
        # decode message
        decoder = decoders[channel.id]
        msg = decoder(encoded_msg.data)
        arr = np.frombuffer(msg.data, np.uint8).reshape(height, width)
        img =  cv2.cvtColor(arr, cv2.COLOR_BayerRG2RGB)
        
        if i == 0:
            pprint(f"{channel.topic} {schema.name}")
            print(f"width: {msg.width}, height: {msg.height}, encoding: {msg.encoding}, step: {msg.step}, shape: {arr.shape}")
        
        # Write video frame
        video.write(img)
        # Save image
        if i % 50 == 0:
            img_path = RGB_IMAGES_PATH / f"{i:0{6}}.png"
        
        cv2.imwrite(str(img_path), img)
        
        if (i % 500 == 0):
            print(f"{i} - {i/num_msgs}")
        i+=1
    video.release()

In [ ]:
# cv2.imshow("test", img)
# cv2.waitKey()
# cv2.destroyAllWindows()
# cv2.imwrite("test.png", img)
img.shape